ALL AGENTING PIPELINES LIKE REACT AND REFLEXTION AGENT

In [31]:
import requests
import datetime
import os
from groq import Groq
from dotenv import load_dotenv
import pickle
from datetime import datetime

load_dotenv()

True

In [12]:
with open('KG/new_graph.pkl', 'rb') as f:
    G = pickle.load(f)

### FUNCTIONS

In [46]:

def search_news(actor, recipient):
    query = f"{actor} and Congress"
    api_key = "fb8156f456be4094b9e88dd55b1d29fa" # Replace with your API key
    url = f"https://newsapi.org/v2/everything?q={query}&sortBy=publishedAt&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        return [{"title": article['title'], "description": article['description'], "url": article['url']} for article in articles[:1]]
    else:
        print(f"Error fetching news articles: {response.status_code}, {response.text}")
        return []


In [3]:

def get_node_edge_connections(graph, actor, recipient, date):
    relevant_edges = []
    relevant_nodes = set()
    for u, v, data in graph.edges(data=True):
        if (actor in [u, v] or recipient in [u, v]) and data.get('relation') == 'event':
            event_date = graph.nodes[u].get('event_date', graph.nodes[v].get('event_date'))
            if event_date and datetime.datetime.strptime(event_date, "%Y-%m-%d") < datetime.datetime.strptime(date, "%Y-%m-%d"):
                relevant_edges.append((u, v, data))
                relevant_nodes.add(u)
                relevant_nodes.add(v)
    return relevant_nodes, relevant_edges


In [45]:

def print_node_attributes(graph, actor, recipient, input_date):
    # Convert input_date to datetime object for comparison
    input_date = datetime.strptime(input_date, "%Y-%m-%d")

    # Create a list to store the events that match the criteria
    matching_events = []

    # Iterate through all nodes in the graph
    for node in graph.nodes:
        # Check if the node has event_id attribute and is an event node
        if 'event_date' in graph.nodes[node]:
            # Get the event date and check if actor and recipient match
            event_date = datetime.strptime(graph.nodes[node]['event_date'], "%Y-%m-%d")
            
            # Find the actor and recipient neighbors
            actor_neighbors = list(graph.neighbors(node))
            recipient_neighbors = list(graph.neighbors(node))
            
            if actor in actor_neighbors and recipient in recipient_neighbors:
                # Calculate the time difference between input date and event date
                time_diff = abs((event_date - input_date).days)
                
                # Append the event along with time difference to the list
                matching_events.append((node, time_diff, graph.nodes[node]))

    # Sort the matching events based on time difference (closest dates first)
    matching_events.sort(key=lambda x: x[1])

    # Select the top 20 events
    top_20_events = matching_events[:5]

    # Return the node attributes for the top 20 events
    top_20_event_data = {event[0]: event[2] for event in top_20_events}
    return top_20_event_data


In [38]:
def calculate_event_type_frequency(graph, actor, recipient, date):
    event_counts = {}
    for u, v, data in graph.edges(data=True):
        if (actor in [u, v] or recipient in [u, v]):
            event_date = graph.nodes[u].get('event_date', graph.nodes[v].get('event_date'))
            if event_date and datetime.strptime(event_date, "%Y-%m-%d") < datetime.strptime(date, "%Y-%m-%d"):
                event_type = graph.nodes[u].get('event_type', graph.nodes[v].get('event_type'))
                if event_type:
                    event_counts[event_type] = event_counts.get(event_type, 0) + 1
    return event_counts


In [47]:
# Initialize Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def query_groq_for_prediction(actor, recipient, date, graph):
    # Step 1: Fetch news articles
    news_articles = search_news(actor, recipient)

    # Step 2: Extract nodes and edges
    nodes, edges = get_node_edge_connections(graph, actor, recipient, date)

    # Step 3: Output node attributes
    node_attributes = print_node_attributes(graph, actor,recipient, date)

    # Step 4: Calculate event type frequency
    event_frequencies = calculate_event_type_frequency(graph, actor, recipient, date)

    # Step 5: Create a structured prompt
    prompt = f"""
You are a knowledge graph analysis expert. Based on the following data, predict the most likely event type between '{actor}' and '{recipient}' on '{date}'.
Recent News Articles:
{news_articles}

*THIS IS KNOWLEDGE GRAPH DATA FOR THE {actor} and {recipient} relations/events
Event Frequency:
{event_frequencies}

Node Attributes:
{node_attributes}

Choose the most likely event type from the following list:
ACCUSE, THREATEN, COERCE, SANCTION, REQUEST, ASSAULT, RETREAT, CONCEDE, MOBILIZE, PROTEST, CONSULT, AID
"""
    print(prompt)
    try:
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="mixtral-8x7b-32768",
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error querying Groq API: {e}")
        return None

In [48]:
# Define the inputs
actor = "Bharatiya Janata Party"
recipient = "Congee"
date = "2024-06-25"

# Run the prediction
prediction = query_groq_for_prediction(actor, recipient, date, G)
print("Predicted Event Type:", prediction)


You are a knowledge graph analysis expert. Based on the following data, predict the most likely event type between 'Bharatiya Janata Party' and 'Congee' on '2024-06-25'.

Recent News Articles:
[{'title': "CNA Explains: What you need to know about India's 'one nation, one election' plan", 'description': "What would the proposal entail, and what are the benefits and drawbacks? CNA's Rohit Rajan breaks it down.", 'url': 'https://www.channelnewsasia.com/asia/india-one-nation-one-election-cna-explains-4813646'}]

Event Frequency:
{'PROTEST': 25, 'THREATEN': 21, 'ACCUSE': 149, 'REQUEST': 17, 'CONCEDE': 22, 'RETREAT': 22, 'MOBILIZE': 26, 'ASSAULT': 4, 'SANCTION': 23, 'COERCE': 10}

Node Attributes:
{'20240619-0830-ca901f31bcb4_ACCUSE': {'event_date': '2024-06-19', 'event_type': 'ACCUSE', 'event_intensity': -3.0, 'quad_code': 'VERBAL CONFLICT', 'contexts': 'election '}, '20240619-5134-c522fb30767d_MOBILIZE': {'event_date': '2024-06-19', 'event_type': 'MOBILIZE', 'event_intensity': -5.5, 'quad

### TOOLS AND AGENT